In [ ]:
# packages
import pandas as pd 
from selenium import webdriver 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm

In [ ]:
# Navigate to the IMDb website
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.imdb.com/search/title/?num_votes=10000,&sort=user_rating,desc&title_type=feature')
driver.implicitly_wait(5)

In [ ]:
# Create empty lists
title = []
link = []
year = []
genre = []

In [ ]:
# First set the total number of films to extract and the number of films per page
total_films_to_extract = 4000
films_per_page = 50

# Calculate the number of pages needed to extract all films
num_pages = total_films_to_extract // films_per_page

for page in tqdm(range(num_pages), desc="Pages Progress"): 
     # Find elements matching movies on the page
    film_elements = driver.find_elements_by_css_selector('.lister-item')
    
    for i in tqdm(range(films_per_page), desc="Films Progress"):  
        #  Extract movie information
        ftitle = film_elements[i].find_element_by_css_selector('.lister-item-header a').text
        fyear = film_elements[i].find_element_by_css_selector('.lister-item-year').text
        flink = film_elements[i].find_element_by_css_selector('.lister-item-header a').get_attribute('href')
        fgenre = film_elements[i].find_element_by_css_selector('.genre').text
        
        # Add extracted information to respective lists
        title.append(ftitle)
        year.append(fyear)
        link.append(flink)
        genre.append(fgenre)
    
    # Move to the next page by clicking the "Next" link
    try:
        next_button = driver.find_element_by_link_text('Next »')
        next_button.click()
    except:
        break  #  Stop if the "Next" link is not found

# Close the browser
driver.quit()

In [ ]:
# create a dataframe
data = {
    'Title': title,
    'Year': year,
    'Link': link,
    'Genre': genre
}
movie_base=pd.DataFrame(data = data)

In [ ]:
# Append new column 'link_review'to the dataframe movie_base
def modify_url(original_url):
    if '?' in original_url:
        modified_url = original_url.replace('?', 'reviews?', 1)
        return modified_url
    else:
        return original_url

movie_base['link_review'] = movie_base['Link'].apply(modify_url)

In [ ]:
# Save the dataframe to a CSV file
movie_base.to_csv('movies.csv', index=False)